# Moroccan real estate market analysis


In [5]:
import pandas as pd

df = pd.read_csv("data/raw-fetched-data.csv")

## Cleaning data

### Removing null values

In [6]:
# Remove columns with more than 60% null values
df.dropna(thresh=df.shape[0] * 0.6, axis=1, inplace=True)

# Remove rows with at least one null value
df.dropna(how='any', axis=0, inplace=True)

In [7]:
# Ensure no null values are left
assert df.isna().sum().sum() == 0

### Cleaning irregular values in ordinal columns

In [8]:
# Entries with more than 10 rooms are represented with 10+ in the ROOMS column
# Replace these values with 11 and set TRUE to the EXTRA_ROOMS column
entries_with_extra_rooms = df["rooms"] == "10+"
df.loc[entries_with_extra_rooms, "rooms"] = "11"

df["EXTRA_ROOMS"] = False
df.loc[entries_with_extra_rooms, "EXTRA_ROOMS"] = True

In [9]:
# Entries with more than 7 bathrooms are represented with 7+ in the BATHROOMS column
# Replace these values with 8 and set TRUE to the EXTRA_BATHROOMS column
entries_with_extra_bathrooms = df["bathrooms"] == "7+"
df.loc[entries_with_extra_bathrooms, "bathrooms"] = "8"

df["EXTRA_BATHROOMS"] = False
df.loc[entries_with_extra_bathrooms, "EXTRA_BATHROOMS"] = True

In [10]:
# Replace these values with 8 and set TRUE to the EXTRA_SPARE_ROOMS column
entries_with_extra_spare_rooms = df["spare_rooms"] == "7+"
df.loc[entries_with_extra_spare_rooms, "spare_rooms"] = "8"

df["EXTRA_SPARE_ROOMS"] = False
df.loc[entries_with_extra_spare_rooms, "EXTRA_SPARE_ROOMS"] = True

In [11]:
# Replace these values with 8 and set TRUE to the EXTRA_FLOOR column
entries_with_extra_floor = df["floor"] == "7+"
df.loc[entries_with_extra_floor, "floor"] = "8"

entries_with_first_floor = df["floor"] == "Rez de chaussée"
df.loc[entries_with_first_floor, "floor"] = "0"

df["EXTRA_FLOOR"] = False
df.loc[entries_with_extra_floor, "EXTRA_FLOOR"] = True

In [12]:
# Ensure no values are left
assert (df["rooms"] == "10+").sum() == 0
assert (df["bathrooms"] == "7+").sum() == 0
assert (df["spare_rooms"] == "7+").sum() == 0
assert (df["floor"] == "7+").sum() == 0
assert (df["floor"] == "Rez de chaussée").sum() == 0

### Converting columns to correct types

In [13]:
# Convert integer based columns
integer_based_columns = [
    'adId', 'listId', 'categoryId', 'parentCategoryId', 'locationCityId', 'locationAreaId', 'spare_rooms', 'floor',
    'rooms', 'bathrooms'
]
df[integer_based_columns] = df[integer_based_columns].astype('int64')

# Convert float based columns
float_based_columns = ['price', 'habitable_size', 'size', 'latitude', 'longitude']
df[float_based_columns] = df[float_based_columns].astype('float64')

# Convert datetime based columns
df['listTime'] = pd.to_datetime(df['listTime'])
df['listTime'] = df['listTime'].dt.floor('min')  # Rounds seconds and nanoseconds to the nearest minute

### Removing outliers

In [14]:
# Removing rows with outliers in the size, habitable size or price column
df = df[
    ((df['price'] > 0) & (df['size'] > 0) & (df['habitable_size'] > 0)) &  # ❗ Update the comment
    (df['size'].quantile(0.05) < df['size']) & (df['size'] < df['size'].quantile(0.95)) &
    (df['habitable_size'].quantile(0.05) < df['habitable_size']) & (df['habitable_size'] < df['habitable_size'].quantile(0.95)) &
    (df['price'].quantile(0.05) < df['price']) & (df['price'] < df['price'].quantile(0.95))
]

In [15]:
df.to_csv("data/processed-data.csv", index=False)